<a href="https://colab.research.google.com/github/LucasAlbFar/Cobertura_Vacinal_BR/blob/main/notebooks/Transforma%C3%A7%C3%A3o_arquivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformação dos arquivos

## Notebook responsável por carregar os arquivos gerados pelo 'Gerar_csv_formatado.ipynb', através do diretório 'dados_carregados;

## Realiza as atualizações de formatação dos arquivos para a análise exploratória dos dados (EDA.ipynb) 

## Arquivos gerados neste notebook são salvos no diretório 'dados_transformados'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def carregar_arquivos(name_files: list, dicionario_arquivos: dict):
  '''
    - Carregar os arquivos em CSV para Pandas DataFrame, contidos no parâmetro 'name_files', no diretório '/dados', para o dicionário 'dicionario_arquivos'
    - Parâmetros: Name Files contém a lista dos nomes dos arquivos a serem transformados em Pandas DataFrame
                  Dicionário Arquivos será o dicionário a ser construído nessa função, contendo o nome do arquivo e seu representação em Pandas DataFrame
  '''

  for name in name_files:
    nome_diretorio = '../content/dados_carregados/' + name + '.csv'
    dicionario_arquivos[name] = pd.read_csv(nome_diretorio)

    index = dicionario_arquivos[name].iloc[:,0].name
    dicionario_arquivos[name] = dicionario_arquivos[name].set_index(index)

In [ ]:
def dataframe_comparativo_regiao_uf_capital(regiao: str, uf: str, capital: str):
  '''
    - Criar Pandas Dataframe para comparar os dados de cobertura vacinal entre um Estado, sua Capital e a sua região geográfica 
    - Parâmetros: Região geográfica do estado e capital
                  UF contendo o estado brasileiro
                  Capital da UF do parâmetro anterior
    - Retorno: Pandas DataFrame
  '''

  df = pd.DataFrame(dicionario_arquivos['ano_por_uf'].T[uf])
  df[capital] = dicionario_arquivos['ano_por_capital'].T[capital]
  df[regiao] = dicionario_arquivos['ano_por_regiao'].T[regiao]
  df = df.reset_index().melt(id_vars=['index'], value_vars=df.columns)
  df.columns = ['ano', 'item', 'cobertura']
  return df

In [ ]:
def dados_transformados_para_csv(dicionario_arquivos: dict):
  '''
    - Salvar cada item contido na lista 'list_file' no diretório '../contente/dados_transformados/'
    - Parâmetros: Dicionario_arquivos contentdo o nome de cada arquivo e o DatFrame Pandas correspondente
  '''

  arquivos = [
    'ano_por_imuno',
    'ano_por_capital',
    'ano_por_uf',
    'ano_por_regiao', 
    'imuno_por_capital',
    'imuno_por_uf',
    'imuno_por_regiao',
    'morbidade_tuberculose',
    'morbidade_meningite',
    'total_por_ano_uf',
    'comparativo_norte',
    'comparativo_nordeste',
    'comparativo_centro_oeste',
    'comparativo_sudeste',
    'comparativo_sul',
    'total_por_enfermidade',
    'total_por_imuno',
    'enfermidades_e_imunos',
    'imunos_e_enfermidades',
    'bcg_tuberculose_ano_a_ano',
    'meningococo_meningite_ano_a_ano',
    'bcg_uf',
    'meningococo_uf',
    'obitos_tuberculose_por_uf',
    'obitos_meningite_por_uf',
    'obitos_tuberculose_comparativo',
    'obitos_meningite_comparativo'
  ]

  for item in arquivos:
    file_name = '../content/dados_transformados/'+ item + '.csv'
    dicionario_arquivos[item].to_csv(file_name)

  print(f'{len(arquivos)} arquivos salvos no diretório dados_transformados.')

In [ ]:
def merge_base_dados(dict_merge: dict, chave: str):
  '''
    - Realizar o merge nos dataframes contidos no dicionário 'dict_merge', pelo parâmetro 'chave
    - Parâmetros: Dict_merge contém os dataframes pandas à serem processados
                  Chave que será utilizada no comando pd.merge
    - Retorno: Pandas DataFrame
  '''

  i = 0
  while (i < (len(dicionario_merge)-1)):
    if (i==0):
      data_frame = list(dicionario_merge.values())[i]

    data_frame = pd.merge(data_frame, list(dicionario_merge.values())[i+1], on=chave)
    i+=1

  return data_frame

In [ ]:
def morbidade_por_enfermidade(enfermidade: str, name_files: dict):
  '''
    - Selecionar os arquivos de determinada enfermidade para serem agreagados através da função 'merge_base_dados'
    - Parâmetros: Enfermidade com o nome a ser pesquisado na lista 'name_files'
                  Name_files com o nome dos arquivos carregados, via URL, pelo notebook 'Gera_csv_formatado'
  '''

  nome_enfermidade = 'morbidade_' + enfermidade

  for item in name_files:
    if nome_enfermidade in item:
      dicionario_merge[item] = dicionario_arquivos[item]

  
  dicionario_arquivos[nome_enfermidade] = merge_base_dados(dicionario_merge, 'Unidade da Federação')
  dicionario_arquivos[nome_enfermidade] = dicionario_arquivos[nome_enfermidade].replace('-', 0)
  dicionario_arquivos[nome_enfermidade] = dicionario_arquivos[nome_enfermidade].astype('int64')


In [ ]:
def comparativo_vacina_obitos(enfermidade: str, imuno: str):
  '''
    - Criar Pandas DataFrame com os dados anuais de óbitos por determinada enfermidade, com a cobertura do imuno vacinal
    - Parâmetros: Enfermidade para acessar os dados contidos em 'dicionario_arquivos'
                  Imuno para acessar os dados contidos em 'dicionario_arquivos['ano_por_imuno']'
  '''

  nome_enfermidade = 'morbidade_' + enfermidade
  df = pd.DataFrame(dicionario_arquivos[nome_enfermidade].sum())
  df = df.reset_index()
  df.columns = ['Ano', 'óbitos']
  df['Ano'] = df['Ano'].astype('int64')
  df = df.set_index('Ano')

  df_dict = pd.DataFrame(dicionario_arquivos['ano_por_imuno'][imuno])
  df_dict = pd.merge(df_dict, df, on='Ano')

  if (enfermidade == 'tuberculose'):
    df_dict = df_dict.iloc[3:,:]

  return df_dict

In [ ]:
def obitos_enfermidade_por_uf(nome_arquivo: str):
  '''
    - Criar Pandas DataFrame com os dados de óbitos de determinada enfermidade em cada UF ao longo do período
    - Parâmetro: Nome_arquivo contido em 'dicionario_arquivos' 
    - Retorno: Pandas DataFrame com o somatório de todos os óbitos de determinada enfermidade
  '''
  df = pd.DataFrame(dicionario_arquivos[nome_arquivo].T.sum())
  df = df.reset_index()
  df.columns = ['uf', 'óbitos']
  df = df.set_index('uf')

  return df


In [ ]:
def cobertura_vacinal_por_uf(imuno: str):
  '''
    - Criar Pandas DataFrame com os dados de cobertura de determinado imuno em cada UF ao longo do período
    - Parâmetro: Imuno contido em 'dicionario_arquivos['imuno_por_uf']' 
    - Retorno: Pandas DataFrame com o somatório da cobertura vacinal de determinado imuno
  '''
  df = pd.DataFrame(data=dicionario_arquivos['imuno_por_uf'][imuno])
  df = df.reset_index()
  df.columns = ['uf', 'cobertura']
  df = df.sort_values(by='cobertura', ascending=False)

  return df

In [ ]:
def comparativo_obitos_uf (enfermidade: str, nome_arquivo: str, lista_estados: list):
  '''
    - Criar Pandas DataFrame com os óbitos por determinada enfermidade, ao longo dos anos
    - Parâmetros: Enfermidade com o nome da enfermidade a ser investigada
                  Nome_arquivo contendo o arquivo a ser recuperados os dados
                  Lista_estados contendo os Estados a serem selecionados para análise
  '''
  df = dicionario_arquivos[nome_arquivo].T[lista_estados]
  
  if (enfermidade == 'tuberculose'):
    df = df.T.drop(['1995', '1996', '1997'], axis=1)
  
  df = df.reset_index().melt(id_vars=['Unidade da Federação'], value_vars=df.columns) 
  df.columns=['uf', 'ano', 'obitos']
  df = df.sort_values(by='obitos', ascending=False)

  return df

In [ ]:
def criar_data_frame(dicionario_dados: dict):
  '''
    - Criar Pandas DataFrame à partir de um dicionário de dados
    - Parâmetros: Dicionario_dados contendo os registros a serem transformados em Pandas DataFrame
  '''
  df = pd.DataFrame(dicionario_dados)
  df = df.T
  df = df.fillna('-')

  return df


In [ ]:
def total_por_categoria(arquivo: pd.DataFrame, categoria: str):
  '''
    - Criar Pandas DataFrame com o total por imuno ou enfermidade 
    - Parâmetros: Arquivo a ser recuperado os registros
                  Categoria à ser analisada, podendo assumir os valores de 'enfermidade' ou 'imuno'
  '''

  if (categoria=='enfermidade'):
    df = pd.DataFrame(arquivo.sum(axis=1))
  else:
    df = pd.DataFrame(arquivo.T.mean(axis=1))

  df = df.reset_index()
  df.columns = [categoria,'cobertura']
  df = df.sort_values('cobertura', ascending=False)

  return df


In [ ]:
# Realizando o carregamento das bases de dados à serem analisadas
name_files = [
  'ano_por_imuno',
  'ano_por_capital',
  'ano_por_uf',
  'ano_por_regiao', 
  'imuno_por_capital',
  'imuno_por_uf',
  'imuno_por_regiao',
  'morbidade_tuberculose_1995_1997_cid09',
  'morbidade_tuberculose_1998_2007_cid10',
  'morbidade_tuberculose_2008_2021',
  'morbidade_meningite_1995_1997_cid09',
  'morbidade_meningite_1998_2007_cid10',
  'morbidade_meningite_2008_2021'
  ]

dicionario_arquivos = {}
carregar_arquivos(name_files,dicionario_arquivos)

In [ ]:
# Criando bases de total de óbitos por enfermidade
dicionario_merge = {}
morbidade_por_enfermidade('tuberculose',name_files)

dicionario_merge = {}
morbidade_por_enfermidade('meningite',name_files)

In [ ]:
# Criando base 'total_por_ano_uf'
dicionario_arquivos['total_por_ano_uf'] = pd.DataFrame(dicionario_arquivos['ano_por_uf'].T.sum(axis=1))

In [ ]:
# Criando base de comparativo da cobertura vacinal entre Região Geográfica, Estado e a Capital desse Estado
dicionario_arquivos['comparativo_norte'] = dataframe_comparativo_regiao_uf_capital('Norte','Amazonas','Manaus')
dicionario_arquivos['comparativo_nordeste'] = dataframe_comparativo_regiao_uf_capital('Nordeste','Ceará','Fortaleza')
dicionario_arquivos['comparativo_centro_oeste'] = dataframe_comparativo_regiao_uf_capital('Centro-Oeste','Mato Grosso','Cuiabá')
dicionario_arquivos['comparativo_sudeste'] = dataframe_comparativo_regiao_uf_capital('Sudeste','Minas Gerais','Belo Horizonte')
dicionario_arquivos['comparativo_sul'] = dataframe_comparativo_regiao_uf_capital('Sul','Rio Grande do Sul','Porto Alegre')

In [ ]:
# Criando base de dados 'enfermidades_e_imunos'
enfermidades_e_imunos = {
        'Tuberculose': {'Imuno_1': 'BCG'},
        'Hepatite A': {'Imuno_1': 'Hepatite A'},
        'Hepatite B': {'Imuno_1': 'Hepatite B', 'Imuno_2': 'Penta', 'Imuno_3': 'Hepatite B em crianças até 30 dias'},
        'Poliomielite': {'Imuno_1': 'Poliomielite', 'Imuno_2': 'Poliomielite(1º ref)', 'Imuno_3': 'Poliomielite 4 anos'},
        'Coqueluche': {'Imuno_1': 'Tetravalente (DTP/Hib) (TETRA)', 'Imuno_2': 'Penta', 'Imuno_3': 'Tríplice Bacteriana(DTP)(1º ref)'},
        'Difteria': {'Imuno_1': 'Tetravalente (DTP/Hib) (TETRA)', 'Imuno_2': 'Penta', 'Imuno_3': 'Tríplice Bacteriana(DTP)(1º ref)', 'Imuno_4': 'Dupla adulto e tríplice acelular gestante'},
        'Tétano': {'Imuno_1': 'Tetravalente (DTP/Hib) (TETRA)', 'Imuno_2': 'Penta', 'Imuno_3': 'dTpa gestante', 'Imuno_4': 'DTP REF (4 e 6 anos)', 'Imuno_5': 'DTP', 'Imuno_6': 'Tríplice Bacteriana(DTP)(1º ref)', 'Imuno_7': 'Dupla adulto e tríplice acelular gestante'},
        'Rubéola': {'Imuno_1': 'Tríplice Viral D1', 'Imuno_2': 'Tríplice Viral D2', 'Imuno_3': 'Tetra Viral(SRC+VZ)'},
        'Influenzae': {'Imuno_1': 'Haemophilus influenzae b', 'Imuno_2': 'Tetravalente (DTP/Hib) (TETRA)', 'Imuno_3': 'Penta'},
        'Rotavírus ': {'Imuno_1': 'Rotavírus Humano'},
        'Febre Amarela': {'Imuno_1': 'Febre Amarela'},
        'Meningite': {'Imuno_1': 'Meningococo C'},
        'Pneumonia': {'Imuno_1': 'Pneumocócica(1º ref)'},
        'Sarampo': {'Imuno_1': 'Tríplice Viral D1', 'Imuno_2': 'Tríplice Viral D2', 'Imuno_3': 'Tetra Viral(SRC+VZ)', 'Imuno_4': 'Sarampo'},
        'Caxumba': {'Imuno_1': 'Tetra Viral(SRC+VZ)'},
        }
        
dicionario_arquivos['enfermidades_e_imunos'] = criar_data_frame(enfermidades_e_imunos)


In [ ]:
# Criando de base de dados relacionando o imuno com as enfermidades que o mesmo protege
imunos_enfermidades = {
        'BCG': {'enfermidade_1': 'Tuberculose'},
        'DTP': {'enfermidade_1': 'Tétano'},
        'DTP REF (4 e 6 anos)': {'enfermidade_1': 'Tétano'},
        'Dupla adulto e tríplice acelular gestante': {'enfermidade_1': 'Tétano', 'enfermidade_2': 'Difteria'},
        'Febre Amarela': {'enfermidade_1': 'Febre_Amarela'},
        'Haemophilus influenzae b': {'enfermidade_1': 'Influenzae'},
        'Hepatite A': {'enfermidade_1': 'Hepatite_A'},
        'Hepatite B': {'enfermidade_1': 'Hepatite_B'},
        'Hepatite B  em crianças até 30 dias': {'enfermidade_1': 'Hepatite_B'},
        'Meningococo C': {'enfermidade_1': 'Meningite'},
        'Penta': {'enfermidade_1': 'Coqueluche', 'enfermidade_2': 'Difteria', 'enfermidade_3': 'Hepatite_B', 'enfermidade_4': 'Influenzae', 'enfermidade_5': 'Tétano'},
        'Pneumocócica(1º ref)': {'enfermidade_1': 'Pneumonia'},
        'Poliomielite': {'enfermidade_1': 'Poliomielite'},
        'Poliomielite 4 anos': {'enfermidade_1': 'Poliomielite'},
        'Poliomielite(1º ref)': {'enfermidade_1': 'Poliomielite'},
        'Rotavírus Humano': {'enfermidade_1': 'Rotavírus'},
        'Sarampo': {'enfermidade_1': 'Sarampo'},
        'Tetra Viral(SRC+VZ)': {'enfermidade_1': 'Caxumba', 'enfermidade_2': 'Rubéola', 'enfermidade_3': 'Sarampo'},
        'Tetravalente (DTP/Hib) (TETRA)': {'enfermidade_1': 'Coqueluche', 'enfermidade_2': 'Difteria', 'enfermidade_3': 'Influenzae', 'enfermidade_4': 'Tétano'},
        'Tríplice Bacteriana(DTP)(1º ref)': {'enfermidade_1': 'Coqueluche', 'enfermidade_2': 'Difteria', 'enfermidade_3': 'Tétano'},
        'Tríplice Viral  D1': {'enfermidade_1': 'Rubéola', 'enfermidade_2': 'Sarampo'},
        'Tríplice Viral  D2': {'enfermidade_1': 'Rubéola', 'enfermidade_2': 'Sarampo'},
        'dTpa gestante': {'enfermidade_1': 'Tétano'}        
        }

dicionario_arquivos['imunos_e_enfermidades'] = criar_data_frame(imunos_enfermidades)

dict_ano_por_enfermidade = {}
cobertura_enfermidade = pd.DataFrame()

for key, value in imunos_enfermidades.items():
  for key_value, value_value in value.items():
    if value_value in cobertura_enfermidade:
      cobertura_enfermidade[value_value] = (cobertura_enfermidade[value_value] + dicionario_arquivos['ano_por_imuno'].T.loc[key]) / 2
    else:
      cobertura_enfermidade[value_value] =  dicionario_arquivos['ano_por_imuno'].T.loc[key]

cobertura_enfermidade = cobertura_enfermidade.T

In [ ]:
# Criando de base de dados para visualizar a cobertura vacinal por enfermidade, ano à ano
dicionario_arquivos['total_por_enfermidade'] = total_por_categoria(cobertura_enfermidade, 'enfermidade')

In [ ]:
# Criando de base de dados para visualizar a cobertura vacinal por imuno
dicionario_arquivos['total_por_imuno'] = total_por_categoria(dicionario_arquivos['ano_por_imuno'], 'imuno')

In [ ]:
# Criando base para a comparação entre cobertura vacinal BCG e óbitos por tuberculose ano à ano
dicionario_arquivos['bcg_tuberculose_ano_a_ano'] = comparativo_vacina_obitos('tuberculose','BCG')
dicionario_arquivos['meningococo_meningite_ano_a_ano'] = comparativo_vacina_obitos('meningite','Meningococo C')

In [ ]:
# Criando base para a comparação entre cobertura vacinal e óbitos por enfermidade nos estados
dicionario_arquivos['obitos_tuberculose_por_uf'] = obitos_enfermidade_por_uf('morbidade_tuberculose')
dicionario_arquivos['obitos_meningite_por_uf'] = obitos_enfermidade_por_uf('morbidade_meningite')

In [ ]:
# Criando base para validar a cobertura vacinal  nas UF's
dicionario_arquivos['bcg_uf'] = cobertura_vacinal_por_uf('BCG')
dicionario_arquivos['meningococo_uf'] = cobertura_vacinal_por_uf('Meningococo C')

In [ ]:
# Criando base para comparação dos óbitos por tuberculose em alguns estados
lista_estados = ['Roraima', 'Pará', 'Rio de Janeiro', 'Mato Grosso', 'Bahia', 'Minas Gerais']
dicionario_arquivos['obitos_tuberculose_comparativo'] = comparativo_obitos_uf('tuberculose', 'morbidade_tuberculose',lista_estados)

In [ ]:
# Criando base para comparação dos óbitos por Infecção Meningocócica em alguns estados
lista_estados = ['São Paulo', 'Goiás', 'Rondônia', 'Sergipe', 'Acre', 'Pará']
dicionario_arquivos['obitos_meningite_comparativo'] = comparativo_obitos_uf('tuberculose', 'morbidade_tuberculose',lista_estados)

In [ ]:
# Enviando os arquivos contidos em 'dicionario_arquivos' para o diretório '/dados_transformados/'
dados_transformados_para_csv(dicionario_arquivos)

27 arquivos salvos no diretório dados_transformados.
